In [52]:
import numpy as np

In [53]:
# class cBoardPosition:
#     connections = None
#     def __init__(self,destType=None, isImmOutsideRoom=False):
#         self.destType=destType
#         self.isImmOutsideRoom = isImmOutsideRoom

In [54]:
class cAmphipod:
    def __init__(self, type="A", position = None):
        self.type = type
        self.position=position
        #self.hasStoppedInHallway = False #maybe dont need this
        #self.hasLeftStartingRoom = False
    
    def __str__(self) -> str:
        #return (f"type={self.type}, position={self.position}, hasStoppedInHallway={self.hasStoppedInHallway}, hasLeftStartingRoom={self.hasLeftStartingRoom}")
        return (f"type={self.type}, position={self.position}")


In [55]:
# #This is probably not a pratical way to handle connections
# class cConnection:
#     #Connection to other nodes
#     def __init__(self, bn1, bn2, steps):
#         self.bn1= bn1
#         self.bn2 = bn2
#         self.steps = steps

Create board positions from input file

In [56]:
filename = "input.txt"
#filename = "input_test0.txt"
#filename = "input_test1.txt"

In [57]:
with open(filename, 'r') as f:
    lines = f.read().splitlines()

In [58]:
lines

['#############',
 '#...........#',
 '###B#B#D#A###',
 '  #C#A#D#C#',
 '  #########']

In [59]:
b_ysize, b_xsize = (len(lines),len(lines[0]))

In [60]:
#Setup board and amphipods
board = np.zeros((len(lines), len(lines[0])), dtype=np.int8)
amphipods = []
#values 0 for wall
#1 space
#2 space but above room

for j in range(len(lines)):
    l0 = lines[j]
    for i in range(len(l0)):
        c0=l0[i]
        if c0=='#' or c0==' ':
            board[j,i] = 0
        else:
            board[j,i]=1
            if j==2:
                board[1,i] =2 #identifies as space just above room
            if c0=='A' or c0=='B' or c0=='C' or c0=='D':
                board[j,i] = 1
                
                a0 = cAmphipod(type=c0, position=[j,i])
                amphipods.append( a0 )

In [61]:
board

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 0],
       [0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int8)

In [62]:
def printAmphipods(amphipods):
    for a in amphipods:
        print(a)

In [63]:
printAmphipods(amphipods)

type=B, position=[2, 3]
type=B, position=[2, 5]
type=D, position=[2, 7]
type=A, position=[2, 9]
type=C, position=[3, 3]
type=A, position=[3, 5]
type=D, position=[3, 7]
type=C, position=[3, 9]


In [64]:
amphipDestinations = { 'A':(3,3) , 'B':(3,5) , 'C':(3,7) , 'D':(3,9)}

In [65]:
def getArrayDistances(type):
    dest = amphipDestinations[type]

    dist_array = np.zeros_like(board)-1 #array with -1
    dist_array[dest[0], dest[1]] = 0
    #print(dist_array)

    hasChanged=True
    while hasChanged:
        hasChanged=False
        for j in range(1, board.shape[0]):
            for i in range(1, board.shape[1]-1):
                if dist_array[j,i]==-1 and board[j,i] >0 :
                    max0 = max(dist_array[j-1,i] , max(dist_array[j+1,i], max(dist_array[j,i-1], dist_array[j,i+1])))
                    
                    if max0>-1:
                        v0 = max0+1
                        dist_array[j,i] = v0
                        hasChanged=True
                        #print(f"j,i = {j},{i} , new value={v0}")
    return dist_array

In [66]:
getArrayDistances('A')

array([[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1,  4,  3,  2,  3,  4,  5,  6,  7,  8,  9, 10, -1],
       [-1, -1, -1,  1, -1,  5, -1,  7, -1,  9, -1, -1, -1],
       [-1, -1, -1,  0, -1,  6, -1,  8, -1, 10, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]], dtype=int8)

In [67]:
arrdist_dict = { 'A' : getArrayDistances('A') ,
    'B':getArrayDistances('B'),
    'C':getArrayDistances('C'),
    'D':getArrayDistances('D')}

In [68]:
arrdist_dict

{'A': array([[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [-1,  4,  3,  2,  3,  4,  5,  6,  7,  8,  9, 10, -1],
        [-1, -1, -1,  1, -1,  5, -1,  7, -1,  9, -1, -1, -1],
        [-1, -1, -1,  0, -1,  6, -1,  8, -1, 10, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]], dtype=int8),
 'B': array([[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [-1,  6,  5,  4,  3,  2,  3,  4,  5,  6,  7,  8, -1],
        [-1, -1, -1,  5, -1,  1, -1,  5, -1,  7, -1, -1, -1],
        [-1, -1, -1,  6, -1,  0, -1,  6, -1,  8, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]], dtype=int8),
 'C': array([[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
        [-1,  8,  7,  6,  5,  4,  3,  2,  3,  4,  5,  6, -1],
        [-1, -1, -1,  7, -1,  5, -1,  1, -1,  5, -1, -1, -1],
        [-1, -1, -1,  8, -1,  6, -1,  0, -1,  6, -1, -1, -1],
        [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]], dtype=int8),
 'D': array([

In [69]:
#Board is the current position of the amphipods
# We can associate a cost when reaching the board state
# We can have a function that calculates the distance score

In [70]:
amphCostMov_dict = { 'A':1 , 'B':10 , 'C':100, 'D':1000 }

In [71]:
board_state = amphipods

In [72]:
def getDistScore(amphipods):
    global arrdist_dict

    total_score=0

    for a0 in amphipods:
        v0 = arrdist_dict[a0.type][a0.position[0], a0.position[1]]
        total_score+= v0
    
    return total_score

In [73]:
getDistScore(board_state)

46

In [74]:
#Check movements
# Loop through all amphipods and check what movements are possible
# Get the scores : movement cost and final board dist score
# Sort and start by choosing the lowestdef

In [75]:
def isOccupied( y,x, amphipods):
    #Check if position is occupied
    for a0 in amphipods:
        a0_y, a0_x = a0.position
        if a0_y == y and a0_x == x:
            return True
    return False

def isNextPosValid(y,x,amphipods):
    #print(f"isNextPosValid y,x = {y},{x}")
    ret=True
    if board[y,x] >0:
        isOcc =  isOccupied(y,x,amphipods)
        #print(f"isOcc={isOcc}")
        if isOcc:
            ret=False
    return ret

def whatAmphipodIsHere(y,x, amphipods):
    #Check if position is occupied and returns the amphipod that occupies it
    # Return None if no amphipods were found
    for a0 in amphipods:
        a0_y, a0_x = a0.position
        if a0_y == y and a0_x == x:
            return a0
    return None


    
def isAmphipodInDestinationRoom(a0):
    destcol = amphipDestinations[a0.type][1]
    if a0.position[1] == destcol and a0.position[0]>=2:
        return True
    return False

def canMoveIntoMyRoom(a0,amphipods):
    #Returns if amphipods can move into its respective room
    #True if room is empty or if room is already filled with correct amphipods

    a0destcol = amphipDestinations[a0.type][1]
    
    for a1 in amphipods:
        if a1.position[1]==a0destcol:
            #is it the same type?
            if a1.type != a0.type:
                return False
    return True

In [76]:
def whatDirections(y,x, amphipods, exclude=None):
    dirs=[]

    if not 'up' in exclude:
        if isNextPosValid(y-1,x, amphipods):
            dirs.append('up')
    if not 'down' in exclude:
        if isNextPosValid(y+1,x, amphipods):
            dirs.append('down')
    if not 'right' in exclude:
        if isNextPosValid(y,x+1, amphipods):
            dirs.append('right')
    if not 'left' in exclude:
        if isNextPosValid(y,x-1, amphipods):
            dirs.append('left')
    return dirs

In [77]:
# Types of moves:
# Leaving from a room and stop somewhere in hallway
# From hallway, move fully into its room, only if room is free
# (Note that leaving room and enter room in one go is a valid move)

In [78]:
def getMovements(amphipods):
    #Returns a list of start and end coordinates
    movements = [] #Start coordinates to end coordinates, and number of steps

    #Check rooms that are incorrectly filled first

    for a0 in amphipods:
        #Check movements
        
        y,x = a0.position
        atype = a0.type

        dest = amphipDestinations[atype]

        #print(f"amphipod y,x {y},{x} ; atype:{atype}")
        
        steps = 0

        #Need to go to halway
        #If in wrong room, yes
        #If in right room, need to check if room is correctly filled

        if y>1:
            #Amphipod is in some room
            # Check if it is in the right room
            # If it is in the right room, check if it has to leave the room to allow others to leave
            
            needToLeaveRoom=False
            if x!= dest[1]:
                #Not in destination room, get out
                needToLeaveRoom=True
            else:
                #It's the right room but check if need to leave
                if not canMoveIntoMyRoom(a0,amphipods):
                    needToLeaveRoom = True

            if needToLeaveRoom:
                #print("needToLeaveRoom")
                #Check access to hallway is clear
                bClear=True
                while y>1 and bClear:
                    y-=1
                    steps+=1
                    if not isNextPosValid(y,x, amphipods):
                        #print("Not a valid position to move to")
                        bClear=False
                
                if bClear:
                    #Check movements to the left and to the right
                    x0=x #backup
                    steps0=steps

                    #left
                    bClear = True
                    while bClear and x>1:
                        x-=1
                        steps+=1
                        if not isNextPosValid(y,x, amphipods):
                            bClear=False
                        else:
                            #Check if can stop here
                            if board[y,x]==1:
                                movements.append([a0.position, [y,x] ,steps])
                
                    #right
                    x=x0
                    steps = steps0
                    bClear = True
                    while bClear and x<b_xsize-1:
                        x+=1
                        steps+=1
                        if not isNextPosValid(y,x, amphipods):
                            bClear=False
                        else:
                            #Check if can stop here
                            if board[y,x]==1:
                                movements.append([a0.position, [y,x] ,steps])

        else:
            #print("Piece in hallway. Move to its respective room if possible")
        
            #Check first if the room is empty or has the correct amphipods already there

            #Move towards the correct column
            
            if canMoveIntoMyRoom(a0,amphipods):
                destcol = dest[1]
                #print("Can move into room")

                xdir = ((destcol - x)>0)*2 -1
                #print(f"xdir:{xdir}")
                stillValid=True
                while stillValid and x!=destcol:
                    x+=xdir
                    steps+=1
                    #Check if there is an amphipod there
                    if isOccupied(y,x,amphipods):
                        stillValid=False
                        break
                #print(f"move-x into room x={x}")
                if stillValid:
                    #Move downwards

                    while stillValid:
                        y0=y+1
                        if isOccupied(y0,x,amphipods) or y0==b_ysize-1:
                            #Set the last position as the final for the movement
                            movements.append([a0.position, [y,x] ,steps])
                            #print(f"move-y into room y={y}")
                            stillValid=False
                        else:
                            y+=1
                            steps+=1

    return movements


In [79]:
#Test
#Amphipod in wrong room
a0 = cAmphipod("A", [2,5])
amhipods1 = [ a0 ]
getMovements(amhipods1)

[[[2, 5], [1, 4], 2],
 [[2, 5], [1, 2], 4],
 [[2, 5], [1, 1], 5],
 [[2, 5], [1, 6], 2],
 [[2, 5], [1, 8], 4],
 [[2, 5], [1, 10], 6],
 [[2, 5], [1, 11], 7]]

In [80]:
#Test
#Amphipod in hallway
a0 = cAmphipod("D", [1,7])
amhipods1 = [ a0 ]
getMovements(amhipods1)
#OK

[[[1, 7], [3, 9], 4]]

In [81]:
#Test
#Amphipod in hallway but blocked by other pieces in room
a0 = cAmphipod("A", [1,7])
a1 = cAmphipod("B", [2,3])
amhipods1 = [ a0 ,a1]
getMovements(amhipods1)
#OK

[[[2, 3], [1, 2], 2],
 [[2, 3], [1, 1], 3],
 [[2, 3], [1, 4], 2],
 [[2, 3], [1, 6], 4]]

In [82]:
# movenodes = []
# for mov0 in getMovements(amphipods):
#     amphipods0 = amphipods.copy() #TODO: check if this is a deep copy
    
#     orig,dest , steps = mov0
#     #Move the amphipod
#     a0 = whatAmphipodIsHere(orig[0], orig[1],amphipods)

#     if not a0 is None:
#         a0.position = dest
    
#         cost = amphCostMov_dict[a0.type]
        
#         scoredist = getDistScore(amphipods0)

#         #Create a new node
#         node0 = node(amphipods0, None, acccost=cost)

#         movenodes.append(node0)

In [83]:
getMovements(amphipods)

[[[2, 3], [1, 2], 2],
 [[2, 3], [1, 1], 3],
 [[2, 3], [1, 4], 2],
 [[2, 3], [1, 6], 4],
 [[2, 3], [1, 8], 6],
 [[2, 3], [1, 10], 8],
 [[2, 3], [1, 11], 9],
 [[2, 5], [1, 4], 2],
 [[2, 5], [1, 2], 4],
 [[2, 5], [1, 1], 5],
 [[2, 5], [1, 6], 2],
 [[2, 5], [1, 8], 4],
 [[2, 5], [1, 10], 6],
 [[2, 5], [1, 11], 7],
 [[2, 7], [1, 6], 2],
 [[2, 7], [1, 4], 4],
 [[2, 7], [1, 2], 6],
 [[2, 7], [1, 1], 7],
 [[2, 7], [1, 8], 2],
 [[2, 7], [1, 10], 4],
 [[2, 7], [1, 11], 5],
 [[2, 9], [1, 8], 2],
 [[2, 9], [1, 6], 4],
 [[2, 9], [1, 4], 6],
 [[2, 9], [1, 2], 8],
 [[2, 9], [1, 1], 9],
 [[2, 9], [1, 10], 2],
 [[2, 9], [1, 11], 3]]

In [84]:
#For each of these movements get the cost and the distance score after the movement
movs = getMovements(amphipods)

In [85]:
class node:
    amphipods = None
    parent_node = None
    acccost = 0
    distance = 0 #automatically calculated
    
    def __init__(self, amphipods, parentnode=None, acccost=0):
        self.amphipods = amphipods
        self.parent_node = parentnode
        self.acccost = acccost
        self.distance = getDistScore(self.amphipods)
    
    def score(self):
        return self.acccost+self.distance
    def score1(self):
        return self.distance
    
    def printStatus(self):
        print(f"Node status. acccost:{self.acccost}, distance:{self.distance}")
        lines=[]
        for i in range(b_ysize):
            lines.append( '#'* b_xsize)
        for y in range(b_ysize):
            for x in range(b_xsize):
                if board[y,x] >0:
                    lines[y] = lines[y][:x] +'.'+ lines[y][x+1:]
        for a0 in self.amphipods:
            y,x = a0.position
            type = a0.type
            lines[y]= lines[y][:x] +type+ lines[y][x+1:]
        for l0 in lines:
            print(l0)
        

In [86]:
def amphDeepCopy(amphipods):
    if amphipods is None:
        return None
    amphs0 = []
    for a0 in amphipods:
        a1 = cAmphipod(a0.type,a0.position.copy())
        amphs0.append(a1)
    return amphs0


In [87]:
# #Test
# amphipods10 = amphDeepCopy(amphipods)
# printAmphipods(amphipods10)
# amphipods10[3].position=[11,11]
# printAmphipods(amphipods)
# printAmphipods(amphipods10)
# #OK

In [88]:
printAmphipods(amphipods)

type=B, position=[2, 3]
type=B, position=[2, 5]
type=D, position=[2, 7]
type=A, position=[2, 9]
type=C, position=[3, 3]
type=A, position=[3, 5]
type=D, position=[3, 7]
type=C, position=[3, 9]


In [89]:
#Create the root node
noderoot = node(amphipods, None, 0)

In [90]:
#Gets movenodes from all the possible movements
movenodes = []
movs = getMovements(amphipods)
for mov0 in movs:
    amphipods0 = amphDeepCopy(amphipods) #TODO: check if this is a deep copy
    
    orig,dest , steps = mov0
    #Move the amphipod
    a0 = whatAmphipodIsHere(orig[0], orig[1], amphipods0)

    if not a0 is None:
        a0.position = dest
    
        cost = amphCostMov_dict[a0.type]*steps
        #TODO: should add to the parent cost
        
        scoredist = getDistScore(amphipods0)

        #Create a new node
        node0 = node(amphipods0, noderoot, cost)

        movenodes.append(node0)

In [91]:
printAmphipods(amphipods)

type=B, position=[2, 3]
type=B, position=[2, 5]
type=D, position=[2, 7]
type=A, position=[2, 9]
type=C, position=[3, 3]
type=A, position=[3, 5]
type=D, position=[3, 7]
type=C, position=[3, 9]


In [92]:
printAmphipods(movenodes[1].amphipods)

type=B, position=[1, 1]
type=B, position=[2, 5]
type=D, position=[2, 7]
type=A, position=[2, 9]
type=C, position=[3, 3]
type=A, position=[3, 5]
type=D, position=[3, 7]
type=C, position=[3, 9]


In [93]:
#Sort moves ascending based on the total score
def nodeScore(node):
    v = node.score1()
    #print(f"node score:{v}")
    return v

In [94]:
movenodes.sort(key = nodeScore) #Sorts in-place

In [95]:
printAmphipods(movenodes[1].amphipods)

type=B, position=[2, 3]
type=B, position=[2, 5]
type=D, position=[2, 7]
type=A, position=[1, 2]
type=C, position=[3, 3]
type=A, position=[3, 5]
type=D, position=[3, 7]
type=C, position=[3, 9]


In [96]:
def checkNodeIsSolution(n1):
    amps=n1.amphipods

    #Check amphipods are all in correct place
    for a0 in amps:
        if not isAmphipodInDestinationRoom(a0):
            return False
    
    return True


In [97]:
solution_nodes = []
best_node = None

In [98]:
#Explore each node
def exploreMovesFromNode(node0, level=0):
    global solution_nodes
    global best_node

    #print(f"Level:{level}")
    #node0.printStatus()
    
    amps0 = node0.amphipods
    movs = getMovements(amps0)
    
    movenodes=None
    
    if len(movs)>0:
        movenodes=[]
        for mov0 in movs:

            amphipods0 = amphDeepCopy(amps0)
    
            orig,dest , steps = mov0
            #Move the amphipod
            a0 = whatAmphipodIsHere(orig[0], orig[1], amphipods0)

            if not a0 is None:
                a0.position = dest
            
                cost = amphCostMov_dict[a0.type]*steps + node0.acccost
                
                #scoredist = getDistScore(amphipods0)

                #Create a new node
                node1 = node(amphipods0, parentnode= node0, acccost=cost)

                movenodes.append(node1)
        
        if len(movenodes)>0:
            movenodes.sort(key = nodeScore) #Sorts in-place

            
            for movn0 in movenodes:
                checkNode=True
                if not best_node is None:
                    if movn0.acccost >= best_node.acccost :
                        checkNode=False
                
                #Check if new node is a solution to the problem
                
                if checkNode:
                    if checkNodeIsSolution(movn0):
                        #Print solution
                        print(f"Solution found at this node acccost:{movn0.acccost} , at level {level}")
                        #solution_nodes.append(movn0)
                        if best_node is None:
                            best_node = movn0
                        else:
                            if movn0.acccost < best_node.acccost:
                                print(f"This is a better solution than previous best node {best_node.acccost}")
                                best_node = movn0
                        
                    else:
                        exploreMovesFromNode(movn0, level+1)


In [99]:
best_node = None
exploreMovesFromNode(noderoot)

Solution found at this node acccost:19646 , at level 15
Solution found at this node acccost:19644 , at level 15
This is a better solution than previous best node 19646
Solution found at this node acccost:11846 , at level 15
This is a better solution than previous best node 19644
Solution found at this node acccost:11844 , at level 15
This is a better solution than previous best node 11846
Solution found at this node acccost:11692 , at level 15
This is a better solution than previous best node 11844
Solution found at this node acccost:11668 , at level 15
This is a better solution than previous best node 11692
Solution found at this node acccost:11666 , at level 15
This is a better solution than previous best node 11668
Solution found at this node acccost:11612 , at level 15
This is a better solution than previous best node 11666
Solution found at this node acccost:11610 , at level 15
This is a better solution than previous best node 11612
Solution found at this node acccost:11608 , at l

KeyboardInterrupt: 

In [ ]:
def printSolution(node):
    nodes = []
    
    while True:
        nodes.insert(0,node)
        node = node.parent_node
        if node is None:
            break
    
    for n0 in nodes:
        n0.printStatus()

In [ ]:
printSolution(best_node)

Node status. acccost:0, distance:30
#############
#...........#
###.#.#D#A###
###.#.#D#A###
#############
Node status. acccost:6, distance:24
#############
#...A.......#
###.#.#D#.###
###.#.#D#A###
#############
Node status. acccost:11, distance:19
#############
#...A.A.....#
###.#.#D#.###
###.#.#D#.###
#############
Node status. acccost:14, distance:16
#############
#.....A.....#
###.#.#D#.###
###A#.#D#.###
#############
Node status. acccost:18, distance:12
#############
#...........#
###A#.#D#.###
###A#.#D#.###
#############
Node status. acccost:2018, distance:10
#############
#.......D...#
###A#.#.#.###
###A#.#D#.###
#############
Node status. acccost:5018, distance:7
#############
#...........#
###A#.#.#.###
###A#.#D#D###
#############
Node status. acccost:8018, distance:4
#############
#.......D...#
###A#.#.#.###
###A#.#.#D###
#############
Node status. acccost:10018, distance:2
#############
#...........#
###A#.#.#D###
###A#.#.#D###
#############
